In [ ]:
import xmltodict
import string

In [ ]:
filename = "/home/santi/workspace/data/pubmed/pubmed18n0929.xml"
with open(filename,"r") as fp:
    data = xmltodict.parse(fp.read())

In [ ]:
printable = set(string.printable)
def get_data(input_file):
    with open(input_file,"r") as fp:
        data = xmltodict.parse(fp.read())
    return data["PubmedArticleSet"]["PubmedArticle"]

def get_text(input_data):
    text= ""
    if isinstance(input_data,dict):
        for key in input_data.keys():
            text += get_text(input_data[key]) 
    elif isinstance(input_data,list):
        for item in input_data:
            text += get_text(item)
    elif isinstance(input_data,str):
        return input_data+" "
    elif input_data:
        out_text = filter(lambda x: x in printable,input_data)
        return out_text+" "
    else:
        return ""
    return text
    

In [ ]:
print(get_text(data["PubmedArticleSet"]["PubmedArticle"][3]).strip())

In [ ]:
from dask.distributed import Client

In [ ]:
client = Client('10.1.10.108:8786')

In [ ]:
data_list = client.map(get_data,[filename])
data_list = client.gather(data_list)

In [ ]:
len(data_list[0])

In [ ]:
text_data = client.map(get_text,data_list[0])
final_text_data = client.gather(text_data)

In [ ]:
len(final_text_data)

In [ ]:
final_text_data[0]

In [ ]:
dirname = "/home/santi/workspace/data/pubmed/abstracts2/"
for _id,article in enumerate(final_text_data):
    filename = dirname + "/abstract_" + str(_id) + ".txt"
    with open(filename,"w") as fp:
        fp.write(article)

In [ ]:
from rocketml.io import DocumentSet
from rocketml.decomposition import TruncatedSVD
from rocketml.preprocessing import TextFilter
from rocketml import Pipeline

In [ ]:
import os
abstract_filenames = [dirname+ff for ff in os.listdir(dirname)]
docset = DocumentSet(filelist=abstract_filenames)
tf = TextFilter(stop_word_filter=True,stemming_filter=True,length_filter=True)
svd = TruncatedSVD(n_components=20)
pipeline = Pipeline([tf,svd])
docset_transofrm = pipeline.fit_transform(docset)
len(docset.documents)

In [ ]:
import pandas as pd
df = pd.DataFrame([[key,value] for key,value in docset.terms_document_frequencies.iteritems()],columns=["Term","Frequency"])
df_high = df[df["Frequency"] > 2000].sort_values(by=["Frequency"],ascending=False)

In [ ]:
terms = sorted(df["Term"])
top_n = 10
comp0 = svd.components_[2]
top_indices = comp0.argsort()[-top_n:][::-1]
for index in top_indices:
    print(terms[index])